In [2]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

In [4]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filtered=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filtered=True)

In [5]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5014.70it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4187.64it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2724164.96it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9091.76it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 4920.66it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 55476.22it/s]


In [6]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3776.78it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00:00, 3218.68it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 69620.96it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 237/237 [00:00<00:00, 6971.39it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2300/2300 [00:00<00:00, 4198.98it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 92324.54it/s]


In [9]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [27]:
bin_size = 30
model = mz.models.ANMM()
# model.params['input_shapes'] = [(5,), (5, 30)]
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['optimizer'] = 'adam'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "input_shapes" set to [(30,), (30,)].
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_right (InputLayer)         (None, 30)           0                                            
__________________________________________________________________________________________________
dropout_7 (Dropout)             (None, 30)           0           text_right[0][0]                 
__________________________________________________________________________________________________
text_left (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
dense_30 (Dense)                (None, 30)           930         dropout_7[0][0]                  
_____________________________________________________________

In [29]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [30]:
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [31]:
model.load_embedding_matrix(embedding_matrix)

In [32]:
pred_generator = mz.HistogramDataGenerator(data_pack=predict_pack_processed,
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, 
                                           x=pred_x, 
                                           y=pred_y, 
                                           once_every=1, 
                                           batch_size=len(pred_y),
                                           model_save_path='./drmm_pretrained_model/'
                                          )

In [33]:
train_generator = mz.HistogramPairDataGenerator(train_pack_processed, embedding_matrix, bin_size, 'LCH',
                                                num_dup=2, num_neg=4, batch_size=20)
len(train_generator)

204

In [34]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30


ValueError: Error when checking input: expected text_left to have shape (30,) but got array with shape (10,)

In [17]:
drmm_model = mz.load_model('./drmm_pretrained_model/16')
test_generator = mz.HistogramDataGenerator(data_pack=valid_pack_processed[:10],
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
test_x, test_y = test_generator[:]
prediction = drmm_model.predict(test_x)
prediction

array([[ 2.0651205 ],
       [-0.8203224 ],
       [ 0.6982938 ],
       [ 0.7515667 ],
       [ 1.9430199 ],
       [-1.5469844 ],
       [-0.62468094],
       [ 0.21224344],
       [-2.4823666 ],
       [-1.5373031 ]], dtype=float32)

In [18]:
import shutil
shutil.rmtree('./drmm_pretrained_model/')